In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten,BatchNormalization
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.callbacks import EarlyStopping

# import warnings
# warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from PIL import Image
import os
from scipy import misc
from  collections import Counter


/home/roman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
PATH_TO_IMAGES="/home/roman/DS_part/image_10_array/"
# PATH_TO_LABELS="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/files/_all_in_one_10.csv"

### Работа с данными

In [31]:
#Загрузка данных
X=np.load(PATH_TO_IMAGES+"X_data.npy")
y=np.load(PATH_TO_IMAGES+"y_data.npy")

In [32]:
#Кодирую имена числами
la=LabelEncoder()
y_copy=y.copy()
y=la.fit_transform(y)

In [33]:
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=17,stratify=y)

In [34]:
num_train, height, width,depth = X_train.shape 
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0] 

#new
X_train =X_train/255 # Normalise data to [0, 1] range
X_test =X_test/255 # Normalise data to [0, 1] range


Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

### Создание архитектуры и обучение модели

In [35]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 30 # we iterate 200 times over the entire training set
kernel_size = (3,3) # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

#### Батч нормализация после relu (специальная начальная инициализация весов)

In [53]:
inp = Input(shape=(height, width,depth))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu',kernel_initializer='he_uniform')(inp)
batch_norm_1=BatchNormalization()(conv_1)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu',kernel_initializer='he_uniform')(batch_norm_1)
batch_norm_2=BatchNormalization()(conv_2)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(batch_norm_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu',kernel_initializer='he_uniform')(drop_1)
batch_norm_3=BatchNormalization()(conv_3)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu',kernel_initializer='he_uniform')(batch_norm_3)
batch_norm_4=BatchNormalization()(conv_4)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(batch_norm_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu',kernel_initializer='he_uniform')(flat)
batch_norm_5=BatchNormalization()(hidden)
drop_3 = Dropout(drop_prob_2)(batch_norm_5)
out = Dense(num_classes, activation='softmax',kernel_initializer='glorot_uniform')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')


model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=2, validation_split=0.1,callbacks=[earlyStopping]) # ...holding out 10% of the data for validation


model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Train on 9903 samples, validate on 1101 samples
Epoch 1/30
 - 71s - loss: 7.7839 - acc: 0.0047 - val_loss: 7.6104 - val_acc: 0.0100
Epoch 2/30
 - 66s - loss: 6.8697 - acc: 0.0392 - val_loss: 7.1480 - val_acc: 0.0400
Epoch 3/30
 - 66s - loss: 5.2055 - acc: 0.1692 - val_loss: 6.2183 - val_acc: 0.1017
Epoch 4/30
 - 66s - loss: 3.3601 - acc: 0.4162 - val_loss: 5.7563 - val_acc: 0.1717
Epoch 5/30
 - 66s - loss: 1.6050 - acc: 0.7297 - val_loss: 4.6658 - val_acc: 0.2843
Epoch 6/30
 - 66s - loss: 0.5343 - acc: 0.9313 - val_loss: 4.8360 - val_acc: 0.2852
4716/4716 [==============================] - 10s 2ms/step


[4.324153309661114, 0.3231552162849873]